In [1]:
import numpy as np
import matplotlib.pyplot as plt
import arrow
import datetime
import xarray as xr
import os
import cmocean.cm as cm
import matplotlib.cm as cma
import scipy.interpolate as interp
from matplotlib.collections import PatchCollection
import matplotlib.dates as mpl_dates
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FormatStrFormatter
import datetime as dt

### Functions

In [2]:
#Function to extract data from 1 section
def get_secdata(datafile,sectionnum):
    
    variables = ['final_salt','final_transp','final_lon','final_lat','final_depth','final_section','final_z','final_temp','init_lon','init_z','init_salt','init_temp','init_transp','final_age']
    
    dict_var = {}
    
    idx = np.where(datafile['final_section']==sectionnum)
    
    for var in variables:
        if var == 'final_age':
            agetd = datafile[var].values
            agehr = agetd.astype('float64')/(10**9*3600)
            age = agehr[idx]
            dict_var[var] = age
        else:
            data_sec = datafile[var]
            dict_var[var] = data_sec[idx]
        
    
    return dict_var

def plotCS(axv,lonsec,zsec,transpsec,lonW,lonE,depi,vmax,time,title,cblabel,ylabel):
    
    cmap = cm.deep
    
    m2lon = ((u_lons[lonE]-u_lons[lonW])/(mesh.e2f[0, latgridi, lonW:lonE].sum())).values #degrees/m
    
    arr,xe,ye,im = axv.hist2d(lonsec/m2lon,df(zsec)[0],weights=transpsec,cmap=cmap,bins=[np.array(f_lons[lonW:lonE:1]/m2lon),w_depths[0,0:depi]],
                              density=True,vmax=vmax
                                )
    
    X,Y = np.meshgrid(xe,ye)
    
    im = axv.pcolormesh(X,Y,arr.T*np.sum(transpsec/time).values,cmap=cmap,vmax=vmax)
    
    axv.invert_yaxis()
    axv.set_title(title)
    axv.set_xlabel('Longitude (°E)')
    axv.set_xticks(np.linspace(f_lons[lonW]/m2lon,f_lons[lonE-1]/m2lon,4))
    axv.xaxis.set_major_formatter(FormatStrFormatter(f"%.{2}f"))
    axv.set_xticklabels(['{:.2f}'.format(label) for label in np.linspace(f_lons[lonW],f_lons[lonE-1],4)])
    # axv.set_xticklabels(np.round(np.linspace(f_lons[lonW],f_lons[lonE-1],4),2))
    
    
    if cblabel == True:
        cb = fig.colorbar(im,ax=axv)
        cb.set_label('Transport (m/s)')
    
    if ylabel == True:
        axv.set_ylabel('Depth (m)')

def plotTS(axn,var,lonsec,zsec,varsec,lonW,lonE,depi,time,title,cblabel,ylabel,vmin=None,vmax=None):
    
    cmap = cm.deep
    
    m2lon = ((u_lons[lonE]-u_lons[lonW])/(mesh.e2f[0, latgridi, lonW:lonE].sum())).values #degrees/m
    
    arrw,xe,ye,im = axn.hist2d(lonsec/m2lon,df(zsec)[0],weights=varsec,cmap=cmap,bins=[np.array(f_lons[lonW:lonE:1]/m2lon),w_depths[0,0:depi]])
    arrnw,xe2,ye2,im2 = axn.hist2d(lonsec/m2lon,df(zsec)[0],cmap=cmap,bins=[np.array(f_lons[lonW:lonE:1]/m2lon),w_depths[0,0:depi]])
    
    arr3 = arrw/arrnw
    
    X,Y = np.meshgrid(xe,ye)
    
    im = axn.pcolormesh(X,Y,arr3.T,vmin=vmin,vmax=vmax)
    axn.set_xlabel('Longitude (°E)')
    axn.set_title(title)
    axn.invert_yaxis()
    axn.set_xticks(np.linspace(f_lons[lonW]/m2lon,f_lons[lonE-1]/m2lon,4))
    axn.set_xticklabels(['{:.2f}'.format(label) for label in np.linspace(f_lons[lonW],f_lons[lonE-1],4)])
    
    if cblabel == True:
        cb = fig.colorbar(im,ax=axn)
        
        if var == 'temp':
            cb.set_label('Temperature (°C)')
        if var == 'sal':
            cb.set_label('Salinity (g/kg)')
            
    if ylabel == True:
        axn.set_ylabel('Depth (m)')
        
def get_datat(datafile,section, starttime, endtime):
    final_section = datafile.final_section
    final_transport = datafile.final_transp
    transports = np.sum(np.where(final_section == section, final_transport, 0)) 
    return transports/(endtime-starttime+1)

def running_mean(data, window_size):
    cumsum = np.cumsum(data)
    cumsum[window_size:] = cumsum[window_size:] - cumsum[:-window_size]
    return cumsum[window_size - 1:] / window_size

In [3]:
data = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/for_jan18/for_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')

lonW = 308
lonE = 327
latgridi = 300
latgridf = latgridi-1
fmask = mesh.fmask[0, :, latgridi]
tmask = mesh.tmask[0]
    
lons = data.init_lon[(data.final_section != 0)]
    
f_lons = mesh.glamf[0, latgridi]
u_lons = mesh.glamv[0, latgridi]
w_depths = mesh.gdepw_1d

m2lon = ((u_lons[lonE]-u_lons[lonW])/(mesh.e2f[0, latgridi, lonW:lonE].sum())).values #degrees/m

df = interp.interp1d(mesh.z,mesh.gdepw_1d) # Interpolate to get same binning scheme as SalishSeaCast

### Importing data

In [112]:
##Forwards - through haro
#Jan
datafvs_haro = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forVStoPR_haro/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafrosS_haro = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forrosS_haro/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafrosN_haro = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forrosN_haro/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafsjcN_haro = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forsjcN_haro/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafsjcS_haro = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forsjcS_haro/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafpr_haro = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forPR_haro/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafadm_haro = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/foradm_haro/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafgulf_haro = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forgulf_haro/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')

#Jul
datafvs_haro07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forVStoPR_haro/for_straight_01jul18_s7t35/ariane_positions_quantitative.nc')
datafpr_haro07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forPR_haro/for_straight_01jul18_s7t35/ariane_positions_quantitative.nc')
datafadm_haro07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/foradm_haro/for_straight_01jul18_s7t35/ariane_positions_quantitative.nc')
datafgulf_haro07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forgulf_haro/for_straight_01jul18_s7t35/ariane_positions_quantitative.nc')

##Forwards - through rosario
datafvs_ros = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forVStoPR_ros/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafharoS_ros = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forharoS_ros/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafharoN_ros = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forharoN_ros/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafsjcN_ros = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forsjcN_ros/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafsjcS_ros = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forsjcS_ros/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafpr_ros = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forpr_ros/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafadm_ros = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/foradm_ros/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafsjcE_ros = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forsjcE_ros/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafgulf_ros = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forgulf_ros/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')

#Jul
datafvs_ros07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forVStoPR_ros/for_straight_01jul18_s7t35/ariane_positions_quantitative.nc')
datafpr_ros07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forpr_ros/for_straight_01jul18_s7t35/ariane_positions_quantitative.nc')
datafadm_ros07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/foradm_ros/for_straight_01jul18_s7t35/ariane_positions_quantitative.nc')
datafgulf_ros07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forgulf_ros/for_straight_01jul18_s7t35/ariane_positions_quantitative.nc')

#Forwards - through sjc
datafvs_sjc = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forVStoPR_sjc/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafpr_sjc = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forpr_sjc/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafadm_sjc = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/foradm_sjc/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')
datafgulf_sjc = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forgulf_sjc/for_straight_01jan18_s7t35/ariane_positions_quantitative.nc')

#Jul
datafvs_sjc07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forVStoPR_sjc/for_straight_01jul18_s7t35/ariane_positions_quantitative.nc')
datafpr_sjc07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forpr_sjc/for_straight_01jul18_s7t35/ariane_positions_quantitative.nc')
datafadm_sjc07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/foradm_sjc/for_straight_01jul18_s7t35/ariane_positions_quantitative.nc')
datafgulf_sjc07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forgulf_sjc/for_straight_01jul18_s7t35/ariane_positions_quantitative.nc')

data_dict = {'datafvs_haro':datafvs_haro,'datafrosS_haro':datafrosS_haro,'datafrosN_haro':datafrosN_haro,
             'datafsjcS_haro':datafsjcS_haro,'datafsjcN_haro':datafsjcN_haro,'datafpr_haro':datafpr_haro,
             'datafadm_haro':datafadm_haro,'datafgulf_haro':datafgulf_haro,'datafvs_ros':datafvs_ros,
             'datafharoS_ros':datafharoS_ros,'datafharoN_ros':datafharoN_ros,
             'datafsjcS_ros':datafsjcS_ros,'datafsjcN_ros':datafsjcN_ros,'datafpr_ros':datafpr_ros,
             'datafadm_ros':datafadm_ros,'datafsjcE_ros':datafsjcE_ros,'datafgulf_ros':datafgulf_ros,'datafpr_sjc':datafpr_sjc,'datafvs_sjc':datafvs_sjc,
             'datafadm_sjc':datafadm_sjc,'datafgulf_sjc':datafgulf_sjc,
             'datafvs_haro07':datafvs_haro07,'datafpr_haro07':datafpr_haro07,'datafadm_haro07':datafadm_haro07,'datafgulf_haro07':datafgulf_haro07,
             'datafvs_ros07':datafvs_ros07,'datafpr_ros07':datafpr_ros07,'datafadm_ros07':datafadm_ros07,'datafgulf_ros07':datafgulf_ros07,
             'datafvs_sjc07':datafvs_sjc07,'datafpr_sjc07':datafpr_sjc07,'datafadm_sjc07':datafadm_sjc07,'datafgulf_sjc07':datafgulf_sjc07
}

In [117]:
starttime = 1
endtime = 168

sectionnum = [0,2,3,4,5,6,7,8,9]

var = ['section_transpfvs_haro','section_transpfrosS_haro','section_transpfrosN_haro','section_transpfsjcS_haro','section_transpfsjcN_haro','section_transpfpr_haro',
       'section_transpfadm_haro','section_transpfgulf_haro','section_transpfvs_ros',
       'section_transpfharoS_ros','section_transpfharoN_ros','section_transpfsjcS_ros','section_transpfsjcN_ros','section_transpfpr_ros',
       'section_transpfadm_ros','section_transpfsjcE_ros','section_transpfgulf_ros','section_transpfvs_sjc','section_transpfpr_sjc','section_transpfadm_sjc','section_transpfgulf_sjc',
       'fvs_haro07','fpr_haro07','fadm_haro07','fgulf_haro07','fvs_ros07','fpr_ros07','fadm_ros07','fgulf_ros07',
       'fvs_sjc07','fpr_sjc07','fadm_sjc07','fgulf_sjc07'
       ]
data_name = ['datafvs_haro','datafrosS_haro','datafrosN_haro','datafsjcS_haro','datafsjcN_haro','datafpr_haro','datafadm_haro','datafgulf_haro','datafvs_ros',
             'datafharoS_ros','datafharoN_ros','datafsjcS_ros','datafsjcN_ros','datafpr_ros','datafadm_ros','datafsjcE_ros','datafgulf_ros','datafvs_sjc','datafpr_sjc','datafadm_sjc','datafgulf_sjc',
             'datafvs_haro07','datafpr_haro07','datafadm_haro07','datafgulf_haro07','datafvs_ros07','datafpr_ros07','datafadm_ros07','datafgulf_ros07','datafvs_sjc07','datafpr_sjc07','datafadm_sjc07','datafgulf_sjc07'
             ]

dict_sectiontransp = {}

for i in range(len(var)):
    variable_name = var[i]
    
    if variable_name not in dict_sectiontransp:
        dict_sectiontransp[variable_name] = []
    
    for section in sectionnum:
        dict_sectiontransp[variable_name].append(get_datat(data_dict[data_name[i]],section,starttime,endtime))


dict_sectiontransp['section_transpfvs_haro']

[586821.0150852797,
 30274.08777043887,
 82.67995097657958,
 6620.950452696759,
 4528.353527573104,
 11407.137016338891,
 2276.9370856140063,
 9784.587192577223,
 8902.40378961045]

### Balance - through haro (Jan)

In [7]:
leave_vic = np.sum(dict_sectiontransp['section_transpfvs_haro'][1:])
ent_vic = np.sum([dict_sectiontransp['section_transpfrosS_haro'][4-1],dict_sectiontransp['section_transpfrosN_haro'][8-1],dict_sectiontransp['section_transpfsjcS_haro'][5-1],dict_sectiontransp['section_transpfsjcN_haro'][9-1],dict_sectiontransp['section_transpfpr_haro'][6-1],dict_sectiontransp['section_transpfadm_haro'][1],dict_sectiontransp['section_transpfgulf_haro'][6]])

leave_rosS = np.sum(dict_sectiontransp['section_transpfrosS_haro'][1:])
ent_rosS = np.sum([dict_sectiontransp['section_transpfrosN_haro'][3],dict_sectiontransp['section_transpfsjcS_haro'][3],dict_sectiontransp['section_transpfsjcN_haro'][3],dict_sectiontransp['section_transpfvs_haro'][3],dict_sectiontransp['section_transpfpr_haro'][3],dict_sectiontransp['section_transpfadm_haro'][3],dict_sectiontransp['section_transpfgulf_haro'][3]])

leave_rosN = np.sum(dict_sectiontransp['section_transpfrosN_haro'][1:])
ent_rosN = np.sum([dict_sectiontransp['section_transpfvs_haro'][7],dict_sectiontransp['section_transpfrosS_haro'][7],dict_sectiontransp['section_transpfsjcS_haro'][7],dict_sectiontransp['section_transpfsjcN_haro'][7],dict_sectiontransp['section_transpfpr_haro'][7],dict_sectiontransp['section_transpfadm_haro'][7],dict_sectiontransp['section_transpfgulf_haro'][7]])

leave_sjcN = np.sum(dict_sectiontransp['section_transpfsjcN_haro'][1:])
ent_sjcN = np.sum([dict_sectiontransp['section_transpfvs_haro'][8],dict_sectiontransp['section_transpfrosS_haro'][8],dict_sectiontransp['section_transpfrosN_haro'][8],dict_sectiontransp['section_transpfsjcS_haro'][8],dict_sectiontransp['section_transpfpr_haro'][8],dict_sectiontransp['section_transpfadm_haro'][8],dict_sectiontransp['section_transpfgulf_haro'][8]])

leave_sjcS = np.sum(dict_sectiontransp['section_transpfsjcS_haro'][1:])
ent_sjcS = np.sum([dict_sectiontransp['section_transpfvs_haro'][4],dict_sectiontransp['section_transpfrosS_haro'][4],dict_sectiontransp['section_transpfrosN_haro'][4],dict_sectiontransp['section_transpfsjcN_haro'][4],dict_sectiontransp['section_transpfpr_haro'][4],dict_sectiontransp['section_transpfadm_haro'][4],dict_sectiontransp['section_transpfgulf_haro'][4]])

leave_pr = np.sum(dict_sectiontransp['section_transpfpr_haro'][1:])
ent_pr = np.sum([dict_sectiontransp['section_transpfvs_haro'][5],dict_sectiontransp['section_transpfrosS_haro'][5],dict_sectiontransp['section_transpfrosN_haro'][5],dict_sectiontransp['section_transpfsjcN_haro'][5],dict_sectiontransp['section_transpfsjcS_haro'][5],dict_sectiontransp['section_transpfadm_haro'][5],dict_sectiontransp['section_transpfgulf_haro'][5]])

leave_adm = np.sum(dict_sectiontransp['section_transpfadm_haro'][1:])
ent_adm = np.sum([dict_sectiontransp['section_transpfvs_haro'][1],dict_sectiontransp['section_transpfrosS_haro'][1],dict_sectiontransp['section_transpfrosN_haro'][1],dict_sectiontransp['section_transpfsjcN_haro'][1],dict_sectiontransp['section_transpfsjcS_haro'][1],dict_sectiontransp['section_transpfadm_haro'][1],dict_sectiontransp['section_transpfgulf_haro'][1]])

leave_gulf = np.sum(dict_sectiontransp['section_transpfgulf_haro'][1:])
ent_gulf = np.sum([dict_sectiontransp['section_transpfvs_haro'][6],dict_sectiontransp['section_transpfrosS_haro'][6],dict_sectiontransp['section_transpfrosN_haro'][6],dict_sectiontransp['section_transpfsjcN_haro'][6],dict_sectiontransp['section_transpfsjcS_haro'][6],dict_sectiontransp['section_transpfadm_haro'][6],dict_sectiontransp['section_transpfgulf_haro'][6]])

leave_total = np.sum([leave_rosS,leave_vic,leave_rosN,leave_sjcS,leave_sjcN,leave_pr,leave_adm,leave_gulf])
ent_total = np.sum([ent_rosS,ent_vic,ent_sjcS,ent_pr,ent_rosN,ent_sjcN,ent_adm,ent_gulf])

print('Leaving VicSill:',leave_vic)
print('Leaving Rosario South:',leave_rosS)
print('Leaving Rosario North:',leave_rosN)
print('Leaving SJC North:',leave_sjcN)
print('Leaving SJC South:',leave_sjcS)
print('Leaving PointRob:',leave_pr)
print('Leaving Admiralty:',leave_adm)
print('Leaving Gulf:',leave_gulf)
print('Total:',leave_total,'\n')

print('Entering Victoria:',ent_vic)
print('Entering Rosario South:',ent_rosS)
print('Entering Rosario North:',ent_rosN)
print('Entering SJC North:',ent_sjcN)
print('Entering SJC South:',ent_sjcS)
print('Entering PointRob:',ent_pr)
print('Entering Admiralty:',ent_adm)
print('Entering Gulf:',ent_gulf)
print('Total:',ent_total)

Leaving VicSill: 73877.13678582589
Leaving Rosario South: 29700.50685782249
Leaving Rosario North: 17424.911073959425
Leaving SJC North: 19852.57276823152
Leaving SJC South: 20241.403913715232
Leaving PointRob: 42429.23740216669
Leaving Admiralty: 45472.464828354714
Leaving Gulf: 13058.496779187268
Total: 262056.73040926323 

Entering Victoria: 79909.29892543705
Entering Rosario South: 13046.202583961493
Entering Rosario North: 35138.00031096685
Entering SJC North: 25357.13634108599
Entering SJC South: 16549.899183679714
Entering PointRob: 38826.51915249656
Entering Admiralty: 76410.46736214634
Entering Gulf: 11198.419628682645
Total: 296435.94348845666


### Balance - through rosario

In [8]:
leave_vic = np.sum(dict_sectiontransp['section_transpfvs_ros'][1:])
ent_vic = np.sum([dict_sectiontransp['section_transpfharoS_ros'][5],dict_sectiontransp['section_transpfharoN_ros'][7],dict_sectiontransp['section_transpfsjcS_ros'][4],dict_sectiontransp['section_transpfsjcN_ros'][8],dict_sectiontransp['section_transpfpr_ros'][3],dict_sectiontransp['section_transpfadm_ros'][1],dict_sectiontransp['section_transpfsjcE_ros'][2]])

leave_haroS = np.sum(dict_sectiontransp['section_transpfharoS_ros'][1:])
ent_haroS = np.sum([dict_sectiontransp['section_transpfvs_ros'][5],dict_sectiontransp['section_transpfharoN_ros'][5],dict_sectiontransp['section_transpfsjcS_ros'][5],dict_sectiontransp['section_transpfsjcN_ros'][5],dict_sectiontransp['section_transpfpr_ros'][5],dict_sectiontransp['section_transpfadm_ros'][5],dict_sectiontransp['section_transpfsjcE_ros'][5]])

leave_haroN = np.sum(dict_sectiontransp['section_transpfharoN_ros'][1:])
ent_haroN = np.sum([dict_sectiontransp['section_transpfharoS_ros'][5],dict_sectiontransp['section_transpfvs_ros'][5],dict_sectiontransp['section_transpfsjcS_ros'][5],dict_sectiontransp['section_transpfsjcN_ros'][5],dict_sectiontransp['section_transpfpr_ros'][5],dict_sectiontransp['section_transpfadm_ros'][5],dict_sectiontransp['section_transpfsjcE_ros'][5]])

leave_sjcN = np.sum(dict_sectiontransp['section_transpfsjcN_ros'][1:])
ent_sjcN = np.sum([dict_sectiontransp['section_transpfharoS_ros'][8],dict_sectiontransp['section_transpfharoN_ros'][8],dict_sectiontransp['section_transpfsjcS_ros'][8],dict_sectiontransp['section_transpfvs_ros'][8],dict_sectiontransp['section_transpfpr_ros'][8],dict_sectiontransp['section_transpfadm_ros'][8],dict_sectiontransp['section_transpfsjcE_ros'][8]])

leave_sjcS = np.sum(dict_sectiontransp['section_transpfsjcS_ros'][1:])
ent_sjcS = np.sum([dict_sectiontransp['section_transpfharoS_ros'][4],dict_sectiontransp['section_transpfharoN_ros'][4],dict_sectiontransp['section_transpfvs_ros'][4],dict_sectiontransp['section_transpfsjcN_ros'][4],dict_sectiontransp['section_transpfpr_ros'][4],dict_sectiontransp['section_transpfadm_ros'][4],dict_sectiontransp['section_transpfsjcE_ros'][4]])

leave_pr = np.sum(dict_sectiontransp['section_transpfpr_ros'][1:])
ent_pr = np.sum([dict_sectiontransp['section_transpfharoS_ros'][3],dict_sectiontransp['section_transpfharoN_ros'][3],dict_sectiontransp['section_transpfsjcS_ros'][3],dict_sectiontransp['section_transpfsjcN_ros'][3],dict_sectiontransp['section_transpfvs_ros'][3],dict_sectiontransp['section_transpfadm_ros'][3],dict_sectiontransp['section_transpfsjcE_ros'][3]])

leave_adm = np.sum(dict_sectiontransp['section_transpfadm_ros'][1:])
ent_adm = np.sum([dict_sectiontransp['section_transpfharoS_ros'][1],dict_sectiontransp['section_transpfharoN_ros'][1],dict_sectiontransp['section_transpfsjcS_ros'][1],dict_sectiontransp['section_transpfsjcN_ros'][1],dict_sectiontransp['section_transpfpr_ros'][1],dict_sectiontransp['section_transpfvs_ros'][1],dict_sectiontransp['section_transpfsjcE_ros'][1]])

leave_sjcE = np.sum(dict_sectiontransp['section_transpfsjcE_ros'][1:])
ent_sjcE = np.sum([dict_sectiontransp['section_transpfharoS_ros'][2],dict_sectiontransp['section_transpfharoN_ros'][2],dict_sectiontransp['section_transpfsjcS_ros'][2],dict_sectiontransp['section_transpfsjcN_ros'][2],dict_sectiontransp['section_transpfpr_ros'][2],dict_sectiontransp['section_transpfadm_ros'][2],dict_sectiontransp['section_transpfvs_ros'][2]])

leave_total = np.sum([leave_haroS,leave_vic,leave_rosN,leave_sjcS,leave_sjcN,leave_pr,leave_adm,leave_sjcE])
ent_total = np.sum([ent_haroS,ent_vic,ent_sjcS,ent_pr,ent_rosN,ent_sjcN,ent_adm,ent_sjcE])

print('Leaving VicSill:',leave_vic)
print('Leaving Haro South:',leave_haroS)
print('Leaving Haro North:',leave_haroN)
print('Leaving SJC North:',leave_sjcN)
print('Leaving SJC South:',leave_sjcS)
print('Leaving PointRob:',leave_pr)
print('Leaving Admiralty:',leave_adm)
print('Leaving SJC East:',leave_sjcE)
print('Total:',leave_total,'\n')

print('Entering Victoria:',ent_vic)
print('Entering Haro South:',ent_rosS)
print('Entering Haro North:',ent_rosN)
print('Entering SJC North:',ent_sjcN)
print('Entering SJC South:',ent_sjcS)
print('Entering PointRob:',ent_pr)
print('Entering Admiralty:',ent_adm)
print('Entering SJC East:',ent_sjcE)
print('Total:',ent_total)

Leaving VicSill: 114316.0161692239
Leaving Haro South: 83130.57901289215
Leaving Haro North: 66866.68323463583
Leaving SJC North: 20491.315477049928
Leaving SJC South: 20631.258550164523
Leaving PointRob: 39916.346023425605
Leaving Admiralty: 45661.30547301323
Leaving SJC East: 1088.5854226449508
Total: 342660.31720237376 

Entering Victoria: 113838.81489232885
Entering Haro South: 13046.202583961493
Entering Haro North: 35138.00031096685
Entering SJC North: 27486.099145457403
Entering SJC South: 17528.349937262617
Entering PointRob: 43806.159120358265
Entering Admiralty: 40643.06126173254
Entering SJC East: 709.9375714363499
Total: 382374.21190624265


### Combined balance

In [10]:
np.sum([dict_sectiontransp['section_transpfpr_ros'][3],dict_sectiontransp['section_transpfadm_ros'][1],dict_sectiontransp['section_transpfpr_haro'][5],dict_sectiontransp['section_transpfadm_haro'][1]])

79631.3133443763

In [27]:
leave_vic = np.sum([np.sum(dict_sectiontransp['section_transpfvs_ros'][1:4]),dict_sectiontransp['section_transpfvs_ros'][6],np.sum(dict_sectiontransp['section_transpfvs_haro'][1:3]),np.sum(dict_sectiontransp['section_transpfvs_haro'][5:7])])
ent_vic = np.sum([dict_sectiontransp['section_transpfpr_ros'][3],dict_sectiontransp['section_transpfadm_ros'][1],dict_sectiontransp['section_transpfpr_haro'][5],dict_sectiontransp['section_transpfadm_haro'][1],dict_sectiontransp['section_transpfgulf_haro'][6],dict_sectiontransp['section_transpfgulf_ros'][6]])

leave_pr = np.sum([np.sum(dict_sectiontransp['section_transpfpr_ros'][1:4]),dict_sectiontransp['section_transpfpr_ros'][6],np.sum(dict_sectiontransp['section_transpfpr_haro'][1:3]),np.sum(dict_sectiontransp['section_transpfpr_haro'][5:7])])
ent_pr = np.sum([dict_sectiontransp['section_transpfvs_ros'][3],dict_sectiontransp['section_transpfadm_ros'][3],dict_sectiontransp['section_transpfvs_haro'][5],dict_sectiontransp['section_transpfadm_haro'][5],dict_sectiontransp['section_transpfgulf_haro'][5],dict_sectiontransp['section_transpfgulf_ros'][3]])

leave_adm = np.sum([np.sum(dict_sectiontransp['section_transpfadm_ros'][1:4]),dict_sectiontransp['section_transpfadm_ros'][6],np.sum(dict_sectiontransp['section_transpfadm_haro'][1:3]),np.sum(dict_sectiontransp['section_transpfadm_haro'][5:7])])
ent_adm = np.sum([dict_sectiontransp['section_transpfvs_ros'][1],dict_sectiontransp['section_transpfpr_ros'][1],dict_sectiontransp['section_transpfvs_haro'][1],dict_sectiontransp['section_transpfpr_haro'][1],dict_sectiontransp['section_transpfgulf_haro'][1],dict_sectiontransp['section_transpfgulf_ros'][1]])

leave_gulf = np.sum([np.sum(dict_sectiontransp['section_transpfgulf_ros'][1:4]),dict_sectiontransp['section_transpfgulf_ros'][6],np.sum(dict_sectiontransp['section_transpfgulf_haro'][1:3]),np.sum(dict_sectiontransp['section_transpfgulf_haro'][5:7])])
ent_gulf = np.sum([dict_sectiontransp['section_transpfvs_ros'][6],dict_sectiontransp['section_transpfpr_ros'][6],dict_sectiontransp['section_transpfvs_haro'][6],dict_sectiontransp['section_transpfpr_haro'][6],dict_sectiontransp['section_transpfadm_haro'][6],dict_sectiontransp['section_transpfadm_ros'][6]])


leave_total = np.sum([leave_vic,leave_pr,leave_adm,leave_gulf])
ent_total = np.sum([ent_vic,ent_pr,ent_adm,ent_gulf])

print('Leaving VicSill:',leave_vic)
# print('Leaving Haro South:',leave_haroS)
# print('Leaving Haro North:',leave_haroN)
# print('Leaving SJC North:',leave_sjcN)
# print('Leaving SJC South:',leave_sjcS)
print('Leaving PointRob:',leave_pr)
print('Leaving Admiralty:',leave_adm)
print('Leaving Gulf:',leave_gulf)
print('Total:',leave_total,'\n')

print('Entering Victoria:',ent_vic)
# print('Entering Haro South:',ent_rosS)
# print('Entering Haro North:',ent_rosN)
# print('Entering SJC North:',ent_sjcN)
# print('Entering SJC South:',ent_sjcS)
print('Entering PointRob:',ent_pr)
print('Entering Admiralty:',ent_adm)
print('Entering Gulf:',ent_gulf)
print('Total:',ent_total)

Leaving VicSill: 75735.1351861342
Leaving PointRob: 27351.04473847934
Leaving Admiralty: 70086.25624834908
Leaving Gulf: 12242.480177832314
Total: 185414.9163507949 

Entering Victoria: 86743.41944928342
Entering PointRob: 18972.13907700795
Entering Admiralty: 62904.003710239274
Entering Gulf: 16428.324201488776
Total: 185047.88643801943


Pretty much balanced

In [42]:
dict_sectiontransp['section_transpfpr_haro'][6],dict_sectiontransp['section_transpfpr_ros'][6]

(7135.570863423385, 6673.935457390362)

In [43]:
dict_sectiontransp['section_transpfvs_haro'][1],dict_sectiontransp['section_transpfvs_ros'][1]

(30274.08777043887, 30975.915698419994)

In [44]:
dict_sectiontransp['section_transpfadm_haro'][1],dict_sectiontransp['section_transpfadm_ros'][1]

(35963.99196220609, 31813.106377097196)

In [45]:
dict_sectiontransp['section_transpfgulf_haro'][5],dict_sectiontransp['section_transpfgulf_ros'][3]

(3001.624340033968, 1998.50592958785)

In [99]:
leave_vic = np.sum([np.mean([dict_sectiontransp['section_transpfvs_ros'][1],dict_sectiontransp['section_transpfvs_haro'][1]]),np.sum(dict_sectiontransp['section_transpfvs_ros'][3]),dict_sectiontransp['section_transpfvs_ros'][6],dict_sectiontransp['section_transpfvs_haro'][2],np.sum(dict_sectiontransp['section_transpfvs_haro'][5:7])])
ent_vic = np.sum([np.mean([dict_sectiontransp['section_transpfadm_ros'][1],dict_sectiontransp['section_transpfadm_haro'][1]]),dict_sectiontransp['section_transpfpr_ros'][3],dict_sectiontransp['section_transpfpr_haro'][5],dict_sectiontransp['section_transpfgulf_haro'][6],dict_sectiontransp['section_transpfgulf_ros'][6]])

leave_pr = np.sum([np.mean([dict_sectiontransp['section_transpfpr_ros'][6],dict_sectiontransp['section_transpfpr_haro'][6]]),np.sum([dict_sectiontransp['section_transpfpr_ros'][1],dict_sectiontransp['section_transpfpr_ros'][3]]),np.sum(dict_sectiontransp['section_transpfpr_haro'][1:3]),np.sum(dict_sectiontransp['section_transpfpr_haro'][5])])
ent_pr = np.sum([np.mean([dict_sectiontransp['section_transpfgulf_ros'][3],dict_sectiontransp['section_transpfgulf_haro'][5]]),dict_sectiontransp['section_transpfvs_ros'][3],dict_sectiontransp['section_transpfadm_ros'][3],dict_sectiontransp['section_transpfvs_haro'][5],dict_sectiontransp['section_transpfadm_haro'][5]])

leave_adm = np.sum([np.mean([dict_sectiontransp['section_transpfadm_ros'][1],dict_sectiontransp['section_transpfadm_haro'][1]]),np.sum(dict_sectiontransp['section_transpfadm_ros'][3]),dict_sectiontransp['section_transpfadm_ros'][6],np.sum(dict_sectiontransp['section_transpfadm_haro'][2]),np.sum(dict_sectiontransp['section_transpfadm_haro'][5:7])])
ent_adm = np.sum([np.mean([dict_sectiontransp['section_transpfvs_haro'][1],dict_sectiontransp['section_transpfvs_ros'][1]]),dict_sectiontransp['section_transpfpr_ros'][1],dict_sectiontransp['section_transpfpr_haro'][1],dict_sectiontransp['section_transpfgulf_haro'][1],dict_sectiontransp['section_transpfgulf_ros'][1]])

leave_gulf = np.sum([np.mean([dict_sectiontransp['section_transpfgulf_ros'][3],dict_sectiontransp['section_transpfgulf_haro'][5]]),np.sum(dict_sectiontransp['section_transpfgulf_ros'][1]),dict_sectiontransp['section_transpfgulf_ros'][6],np.sum(dict_sectiontransp['section_transpfgulf_haro'][2]),np.sum(dict_sectiontransp['section_transpfgulf_haro'][5:7])])
ent_gulf = np.sum([np.mean([dict_sectiontransp['section_transpfpr_haro'][6],dict_sectiontransp['section_transpfpr_ros'][6]]),dict_sectiontransp['section_transpfvs_haro'][6],dict_sectiontransp['section_transpfvs_ros'][6],dict_sectiontransp['section_transpfadm_haro'][6],dict_sectiontransp['section_transpfadm_ros'][6]])


leave_total = np.sum([leave_vic,leave_pr,leave_adm,leave_gulf])
ent_total = np.sum([ent_vic,ent_pr,ent_adm,ent_gulf])

print('Leaving VicSill:',leave_vic)
print('Leaving PointRob:',leave_pr)
print('Leaving Admiralty:',leave_adm)
print('Leaving Gulf:',leave_gulf)
print('Total:',leave_total,'\n')

print('Entering Victoria:',ent_vic)
print('Entering PointRob:',ent_pr)
print('Entering Admiralty:',ent_adm)
print('Entering Gulf:',ent_gulf)
print('Total:',ent_total)

Leaving VicSill: 45041.47509878072
Leaving PointRob: 20286.854994082823
Leaving Admiralty: 36159.03850349755
Leaving Gulf: 12649.254611236094
Total: 114136.62320759718 

Entering Victoria: 52854.87027963179
Entering PointRob: 16472.07394219704
Entering Admiralty: 32279.001975809842
Entering Gulf: 9523.571041081901
Total: 111129.51723872057


Slightly out of balance ... because no seeding at SJCEast? Or could be because I'm averaging what doesn't pass through a strait

I think its because I'm averaging since it was mostly balances before (above)

In [89]:
N_haro = np.sum([dict_sectiontransp['section_transpfvs_haro'][5:7],dict_sectiontransp['section_transpfadm_haro'][5:7]])
N_ros = np.sum([dict_sectiontransp['section_transpfvs_ros'][3],dict_sectiontransp['section_transpfadm_ros'][3],dict_sectiontransp['section_transpfvs_ros'][6],dict_sectiontransp['section_transpfadm_ros'][6]])

print(N_haro,N_ros)
print(ent_gulf+ent_pr)
#Check that whats going through each strait + whats going b/w pr and gulf is =
total = N_haro+N_ros+np.mean([dict_sectiontransp['section_transpfpr_haro'][6],dict_sectiontransp['section_transpfpr_ros'][6]])+np.mean([dict_sectiontransp['section_transpfgulf_haro'][5],dict_sectiontransp['section_transpfgulf_ros'][3]])
print(total)


15696.073317048913 894.753371012245
25995.64498327894
25995.644983278944


The total of whats entering gulf + pr is equal to what is passing through rosario and haro and whats moving between the two --> balances!

In [91]:
S_haro = np.sum([dict_sectiontransp['section_transpfpr_haro'][5],dict_sectiontransp['section_transpfpr_haro'][1],dict_sectiontransp['section_transpfgulf_haro'][6],dict_sectiontransp['section_transpfgulf_haro'][1]])
S_ros = np.sum([dict_sectiontransp['section_transpfpr_ros'][3],dict_sectiontransp['section_transpfpr_ros'][1],dict_sectiontransp['section_transpfgulf_ros'][6],dict_sectiontransp['section_transpfgulf_ros'][1]])

print(S_haro,S_ros)
print(ent_vic+ent_adm)
#Check that whats going passing south through the channels (+ whats moving b/w them) is = to whats entering adm + vs
total = S_haro+S_ros+np.mean([dict_sectiontransp['section_transpfadm_haro'][1],dict_sectiontransp['section_transpfadm_ros'][1]])+np.mean([dict_sectiontransp['section_transpfvs_haro'][1],dict_sectiontransp['section_transpfvs_ros'][1]])
print(total)

18615.04547401769 2005.2758773428677
85133.87225544163
85133.87225544163


Balance --> good

### Balance with SJC

In [105]:
leave_vic = np.sum([np.mean([dict_sectiontransp['section_transpfvs_ros'][1],dict_sectiontransp['section_transpfvs_haro'][1],dict_sectiontransp['section_transpfvs_sjc'][1]]),np.sum(dict_sectiontransp['section_transpfvs_ros'][3]),dict_sectiontransp['section_transpfvs_ros'][6],dict_sectiontransp['section_transpfvs_haro'][2],np.sum(dict_sectiontransp['section_transpfvs_haro'][5:7]),dict_sectiontransp['section_transpfvs_sjc'][4],dict_sectiontransp['section_transpfvs_sjc'][6]])
ent_vic = np.sum([np.mean([dict_sectiontransp['section_transpfadm_ros'][1],dict_sectiontransp['section_transpfadm_haro'][1],dict_sectiontransp['section_transpfadm_sjc'][1]]),dict_sectiontransp['section_transpfpr_ros'][3],dict_sectiontransp['section_transpfpr_haro'][5],dict_sectiontransp['section_transpfgulf_haro'][6],dict_sectiontransp['section_transpfgulf_ros'][6],dict_sectiontransp['section_transpfpr_sjc'][4],dict_sectiontransp['section_transpfgulf_sjc'][6]])

leave_pr = np.sum([np.mean([dict_sectiontransp['section_transpfpr_ros'][6],dict_sectiontransp['section_transpfpr_haro'][6],dict_sectiontransp['section_transpfpr_sjc'][6]]),np.sum([dict_sectiontransp['section_transpfpr_ros'][1],dict_sectiontransp['section_transpfpr_ros'][3]]),np.sum(dict_sectiontransp['section_transpfpr_haro'][1:3]),np.sum(dict_sectiontransp['section_transpfpr_haro'][5]),dict_sectiontransp['section_transpfpr_sjc'][1],dict_sectiontransp['section_transpfpr_sjc'][4]])
ent_pr = np.sum([np.mean([dict_sectiontransp['section_transpfgulf_ros'][3],dict_sectiontransp['section_transpfgulf_haro'][5],dict_sectiontransp['section_transpfgulf_sjc'][4]]),dict_sectiontransp['section_transpfvs_ros'][3],dict_sectiontransp['section_transpfadm_ros'][3],dict_sectiontransp['section_transpfvs_haro'][5],dict_sectiontransp['section_transpfadm_haro'][5],dict_sectiontransp['section_transpfvs_sjc'][4],dict_sectiontransp['section_transpfadm_sjc'][4]])

leave_adm = np.sum([np.mean([dict_sectiontransp['section_transpfadm_ros'][1],dict_sectiontransp['section_transpfadm_haro'][1],dict_sectiontransp['section_transpfadm_sjc'][1]]),np.sum(dict_sectiontransp['section_transpfadm_ros'][3]),dict_sectiontransp['section_transpfadm_ros'][6],np.sum(dict_sectiontransp['section_transpfadm_haro'][2]),np.sum(dict_sectiontransp['section_transpfadm_haro'][5:7]),dict_sectiontransp['section_transpfadm_sjc'][4],dict_sectiontransp['section_transpfadm_sjc'][6]])
ent_adm = np.sum([np.mean([dict_sectiontransp['section_transpfvs_haro'][1],dict_sectiontransp['section_transpfvs_ros'][1],dict_sectiontransp['section_transpfvs_sjc'][1]]),dict_sectiontransp['section_transpfpr_ros'][1],dict_sectiontransp['section_transpfpr_haro'][1],dict_sectiontransp['section_transpfgulf_haro'][1],dict_sectiontransp['section_transpfgulf_ros'][1],dict_sectiontransp['section_transpfpr_sjc'][1],dict_sectiontransp['section_transpfgulf_sjc'][1]])

leave_gulf = np.sum([np.mean([dict_sectiontransp['section_transpfgulf_ros'][3],dict_sectiontransp['section_transpfgulf_haro'][5],dict_sectiontransp['section_transpfgulf_sjc'][4]]),np.sum(dict_sectiontransp['section_transpfgulf_ros'][1]),dict_sectiontransp['section_transpfgulf_ros'][6],np.sum(dict_sectiontransp['section_transpfgulf_haro'][2]),np.sum(dict_sectiontransp['section_transpfgulf_haro'][5:7]),dict_sectiontransp['section_transpfgulf_sjc'][1],dict_sectiontransp['section_transpfgulf_sjc'][6]])
ent_gulf = np.sum([np.mean([dict_sectiontransp['section_transpfpr_haro'][6],dict_sectiontransp['section_transpfpr_ros'][6],dict_sectiontransp['section_transpfpr_sjc'][6]]),dict_sectiontransp['section_transpfvs_haro'][6],dict_sectiontransp['section_transpfvs_ros'][6],dict_sectiontransp['section_transpfadm_haro'][6],dict_sectiontransp['section_transpfadm_ros'][6],dict_sectiontransp['section_transpfvs_sjc'][6],dict_sectiontransp['section_transpfadm_sjc'][6]])

leave_total = np.sum([leave_vic,leave_pr,leave_adm,leave_gulf])
ent_total = np.sum([ent_vic,ent_pr,ent_adm,ent_gulf])

print('Leaving VicSill:',leave_vic)
print('Leaving PointRob:',leave_pr)
print('Leaving Admiralty:',leave_adm)
print('Leaving Gulf:',leave_gulf)
print('Total:',leave_total,'\n')

print('Entering Victoria:',ent_vic)
print('Entering PointRob:',ent_pr)
print('Entering Admiralty:',ent_adm)
print('Entering Gulf:',ent_gulf)
print('Total:',ent_total)

Leaving VicSill: 44858.46883947505
Leaving PointRob: 20943.22905230422
Leaving Admiralty: 35584.72844506591
Leaving Gulf: 12511.162990821244
Total: 113897.58932766643 

Entering Victoria: 52914.09829229986
Entering PointRob: 16397.497241369478
Entering Admiralty: 32079.868449673144
Entering Gulf: 9499.019375447326
Total: 110890.48335878982


Look at how different vs to adm is b/w _ros, _haro and _sjc:

In [98]:
dict_sectiontransp['section_transpfvs_haro'][1],dict_sectiontransp['section_transpfvs_ros'][1],dict_sectiontransp['section_transpfvs_sjc'][1]

(30274.08777043887, 30975.915698419994, 29698.89569663809)

In [104]:
dict_sectiontransp['section_transpfpr_haro'][6],dict_sectiontransp['section_transpfpr_ros'][6],dict_sectiontransp['section_transpfpr_sjc'][6]

(7135.570863423385, 6673.935457390362, 6722.255175039302)

In [109]:
N_haro = np.sum([dict_sectiontransp['section_transpfvs_haro'][5:7],dict_sectiontransp['section_transpfadm_haro'][5:7]])
N_ros = np.sum([dict_sectiontransp['section_transpfvs_ros'][3],dict_sectiontransp['section_transpfadm_ros'][3],dict_sectiontransp['section_transpfvs_ros'][6],dict_sectiontransp['section_transpfadm_ros'][6]])
N_sjc = np.sum([dict_sectiontransp['section_transpfvs_sjc'][4],dict_sectiontransp['section_transpfadm_sjc'][4],dict_sectiontransp['section_transpfvs_sjc'][6],dict_sectiontransp['section_transpfadm_sjc'][6]])

print(N_haro,N_ros,N_sjc)
print(ent_gulf+ent_pr)
#Check that whats going through each strait + whats going b/w pr and gulf is =
total = N_haro+N_ros+N_sjc+np.mean([dict_sectiontransp['section_transpfpr_haro'][6],dict_sectiontransp['section_transpfpr_ros'][6],dict_sectiontransp['section_transpfpr_sjc'][6]])+np.mean([dict_sectiontransp['section_transpfgulf_haro'][5],dict_sectiontransp['section_transpfgulf_ros'][3],dict_sectiontransp['section_transpfgulf_sjc'][4]])
print(total)

15696.073317048913 894.753371012245 154.86097023821574
25896.516616816803
25896.516616816807


In [111]:
S_haro = np.sum([dict_sectiontransp['section_transpfpr_haro'][5],dict_sectiontransp['section_transpfpr_haro'][1],dict_sectiontransp['section_transpfgulf_haro'][6],dict_sectiontransp['section_transpfgulf_haro'][1]])
S_ros = np.sum([dict_sectiontransp['section_transpfpr_ros'][3],dict_sectiontransp['section_transpfpr_ros'][1],dict_sectiontransp['section_transpfgulf_ros'][6],dict_sectiontransp['section_transpfgulf_ros'][1]])
S_sjc = np.sum([dict_sectiontransp['section_transpfpr_sjc'][4],dict_sectiontransp['section_transpfpr_sjc'][1],dict_sectiontransp['section_transpfgulf_sjc'][6],dict_sectiontransp['section_transpfgulf_sjc'][1]])

print(S_haro,S_ros,S_sjc)
print(ent_vic+ent_adm)
#Check that whats going passing south through the channels (+ whats moving b/w them) is = to whats entering adm + vs
total = S_haro+S_ros+S_sjc+np.mean([dict_sectiontransp['section_transpfadm_haro'][1],dict_sectiontransp['section_transpfadm_ros'][1],dict_sectiontransp['section_transpfadm_sjc'][1]])+np.mean([dict_sectiontransp['section_transpfvs_haro'][1],dict_sectiontransp['section_transpfvs_ros'][1],dict_sectiontransp['section_transpfvs_sjc'][1]])
print(total)

18615.04547401769 2005.2758773428677 772.271774506898
84993.96674197301
84993.96674197301


### Balance in July 

In [119]:
leave_vic = np.sum([np.mean([dict_sectiontransp['fvs_ros07'][1],dict_sectiontransp['fvs_haro07'][1],dict_sectiontransp['fvs_sjc07'][1]]),np.sum(dict_sectiontransp['fvs_ros07'][3]),dict_sectiontransp['fvs_ros07'][6],dict_sectiontransp['fvs_haro07'][2],np.sum(dict_sectiontransp['fvs_haro07'][5:7]),dict_sectiontransp['fvs_sjc07'][4],dict_sectiontransp['fvs_sjc07'][6]])
ent_vic = np.sum([np.mean([dict_sectiontransp['fadm_ros07'][1],dict_sectiontransp['fadm_haro07'][1],dict_sectiontransp['fadm_sjc07'][1]]),dict_sectiontransp['fpr_ros07'][3],dict_sectiontransp['fpr_haro07'][5],dict_sectiontransp['fgulf_haro07'][6],dict_sectiontransp['fgulf_ros07'][6],dict_sectiontransp['fpr_sjc07'][4],dict_sectiontransp['fgulf_sjc07'][6]])

leave_pr = np.sum([np.mean([dict_sectiontransp['fpr_ros07'][6],dict_sectiontransp['fpr_haro07'][6],dict_sectiontransp['fpr_sjc07'][6]]),np.sum([dict_sectiontransp['fpr_ros07'][1],dict_sectiontransp['fpr_ros07'][3]]),np.sum(dict_sectiontransp['fpr_haro07'][1:3]),np.sum(dict_sectiontransp['fpr_haro07'][5]),dict_sectiontransp['fpr_sjc07'][1],dict_sectiontransp['fpr_sjc07'][4]])
ent_pr = np.sum([np.mean([dict_sectiontransp['fgulf_ros07'][3],dict_sectiontransp['fgulf_haro07'][5],dict_sectiontransp['fgulf_sjc07'][4]]),dict_sectiontransp['fvs_ros07'][3],dict_sectiontransp['fadm_ros07'][3],dict_sectiontransp['fvs_haro07'][5],dict_sectiontransp['fadm_haro07'][5],dict_sectiontransp['fvs_sjc07'][4],dict_sectiontransp['fadm_sjc07'][4]])

leave_adm = np.sum([np.mean([dict_sectiontransp['fadm_ros07'][1],dict_sectiontransp['fadm_haro07'][1],dict_sectiontransp['fadm_sjc07'][1]]),np.sum(dict_sectiontransp['fadm_ros07'][3]),dict_sectiontransp['fadm_ros07'][6],np.sum(dict_sectiontransp['fadm_haro07'][2]),np.sum(dict_sectiontransp['fadm_haro07'][5:7]),dict_sectiontransp['fadm_sjc07'][4],dict_sectiontransp['fadm_sjc07'][6]])
ent_adm = np.sum([np.mean([dict_sectiontransp['fvs_haro07'][1],dict_sectiontransp['fvs_ros07'][1],dict_sectiontransp['fvs_sjc07'][1]]),dict_sectiontransp['fpr_ros07'][1],dict_sectiontransp['fpr_haro07'][1],dict_sectiontransp['fgulf_haro07'][1],dict_sectiontransp['fgulf_ros07'][1],dict_sectiontransp['fpr_sjc07'][1],dict_sectiontransp['fgulf_sjc07'][1]])

leave_gulf = np.sum([np.mean([dict_sectiontransp['fgulf_ros07'][3],dict_sectiontransp['fgulf_haro07'][5],dict_sectiontransp['fgulf_sjc07'][4]]),np.sum(dict_sectiontransp['fgulf_ros07'][1]),dict_sectiontransp['fgulf_ros07'][6],np.sum(dict_sectiontransp['fgulf_haro07'][2]),np.sum(dict_sectiontransp['fgulf_haro07'][5:7]),dict_sectiontransp['fgulf_sjc07'][1],dict_sectiontransp['fgulf_sjc07'][6]])
ent_gulf = np.sum([np.mean([dict_sectiontransp['fpr_haro07'][6],dict_sectiontransp['fpr_ros07'][6],dict_sectiontransp['fpr_sjc07'][6]]),dict_sectiontransp['fvs_haro07'][6],dict_sectiontransp['fvs_ros07'][6],dict_sectiontransp['fadm_haro07'][6],dict_sectiontransp['fadm_ros07'][6],dict_sectiontransp['fvs_sjc07'][6],dict_sectiontransp['fadm_sjc07'][6]])

leave_total = np.sum([leave_vic,leave_pr,leave_adm,leave_gulf])
ent_total = np.sum([ent_vic,ent_pr,ent_adm,ent_gulf])

print('Leaving VicSill:',leave_vic)
print('Leaving PointRob:',leave_pr)
print('Leaving Admiralty:',leave_adm)
print('Leaving Gulf:',leave_gulf)
print('Total:',leave_total,'\n')

print('Entering Victoria:',ent_vic)
print('Entering PointRob:',ent_pr)
print('Entering Admiralty:',ent_adm)
print('Entering Gulf:',ent_gulf)
print('Total:',ent_total)

Leaving VicSill: 65063.42935173435
Leaving PointRob: 41078.05040376207
Leaving Admiralty: 36825.41005582547
Leaving Gulf: 12560.79453643553
Total: 155527.68434775743 

Entering Victoria: 74825.08842276865
Entering PointRob: 37920.3650987503
Entering Admiralty: 31195.65043445807
Entering Gulf: 9532.525863510917
Total: 153473.62981948792


In [120]:
N_haro07 = np.sum([dict_sectiontransp['fvs_haro07'][5:7],dict_sectiontransp['fadm_haro07'][5:7]])
N_ros07 = np.sum([dict_sectiontransp['fvs_ros07'][3],dict_sectiontransp['fadm_ros07'][3],dict_sectiontransp['fvs_ros07'][6],dict_sectiontransp['fadm_ros07'][6]])
N_sjc07 = np.sum([dict_sectiontransp['fvs_sjc07'][4],dict_sectiontransp['fadm_sjc07'][4],dict_sectiontransp['fvs_sjc07'][6],dict_sectiontransp['fadm_sjc07'][6]])

print(N_haro07,N_ros07,N_sjc07)
print(ent_gulf+ent_pr)
#Check that whats going through each strait + whats going b/w pr and gulf is =
total = N_haro07+N_ros07+N_sjc07+np.mean([dict_sectiontransp['fpr_haro07'][6],dict_sectiontransp['fpr_ros07'][6],dict_sectiontransp['fpr_sjc07'][6]])+np.mean([dict_sectiontransp['fgulf_haro07'][5],dict_sectiontransp['fgulf_ros07'][3],dict_sectiontransp['fgulf_sjc07'][4]])
print(total)

39445.37634590847 198.62851402392297 46.72469457323586
47452.89096226121
47452.8909622612


In [121]:
S_haro07 = np.sum([dict_sectiontransp['fpr_haro07'][5],dict_sectiontransp['fpr_haro07'][1],dict_sectiontransp['fgulf_haro07'][6],dict_sectiontransp['fgulf_haro07'][1]])
S_ros07 = np.sum([dict_sectiontransp['fpr_ros07'][3],dict_sectiontransp['fpr_ros07'][1],dict_sectiontransp['fgulf_ros07'][6],dict_sectiontransp['fgulf_ros07'][1]])
S_sjc07 = np.sum([dict_sectiontransp['fpr_sjc07'][4],dict_sectiontransp['fpr_sjc07'][1],dict_sectiontransp['fgulf_sjc07'][6],dict_sectiontransp['fgulf_sjc07'][1]])

print(S_haro07,S_ros07,S_sjc07)
print(ent_vic+ent_adm)
#Check that whats going passing south through the channels (+ whats moving b/w them) is = to whats entering adm + vs
total = S_haro07+S_ros07+S_sjc07+np.mean([dict_sectiontransp['fadm_haro07'][1],dict_sectiontransp['fadm_ros07'][1],dict_sectiontransp['fadm_sjc07'][1]])+np.mean([dict_sectiontransp['fvs_haro07'][1],dict_sectiontransp['fvs_ros07'][1],dict_sectiontransp['fvs_sjc07'][1]])
print(total)

27684.683459838558 13761.186639120797 2460.9412581613547
106020.73885722672
106020.73885722674


Look @ property modification through haro vs. through Rosario